In [1]:
import torch
from torch.utils.data import Dataset,DataLoader


In [2]:
# Step 1: Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        """
        Args:
            data (list): List of input data.
            targets (list): List of corresponding labels/targets.
        """
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns a single sample and its corresponding target.
        """
        sample = self.data[idx]
        target = self.targets[idx]
        return sample, target


In [3]:
# Step 2: Create dummy data
data = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]])
targets = torch.tensor([0, 1, 0, 1])

In [4]:
# Step 3: Create a dataset instance
dataset = CustomDataset(data, targets)

In [5]:
# Step 4: Create a DataLoader
dataloader = DataLoader(
    dataset,          # Dataset to load
    batch_size=2,     # Number of samples per batch
    shuffle=True,     # Shuffle the data
    num_workers=0     # Number of subprocesses for data loading (0 means no parallel loading)
)

In [6]:
# Step 5: Iterate over the DataLoader
for batch_idx, (batch_data, batch_targets) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}:")
    print(f"Data: {batch_data}")
    print(f"Targets: {batch_targets}")
    print()

Batch 1:
Data: tensor([[3., 4.],
        [5., 6.]])
Targets: tensor([1, 0])

Batch 2:
Data: tensor([[7., 8.],
        [1., 2.]])
Targets: tensor([1, 0])



In [7]:
import torch.nn as nn
# Define input (batch size = 1, channels = 1, height = 5, width = 5)
input = torch.tensor([[
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]
]], dtype=torch.float32).unsqueeze(0)  # Add batch and channel dimensions
print(input.shape)

torch.Size([1, 1, 5, 5])


In [10]:
# Define a convolution layer
conv_layer = nn.Conv2d(  # preferred in encoder
    in_channels=1,       # Input channels
    out_channels=1,      # Output channels
    kernel_size=3,       # Kernel size (3x3)
    stride=1,            # Stride
    padding=0            # No padding
)

In [9]:
# Define a deconvolution (transposed convolution) layer
deconv_layer = nn.ConvTranspose2d( # preferred in decoder
    in_channels=1,       # Input channels
    out_channels=1,      # Output channels
    kernel_size=3,       # Kernel size (3x3)
    stride=1,            # Stride
    padding=0            # No padding
)

In [11]:
# Perform convolution
output_conv = conv_layer(input)
print("Convolution Output:")
print(output_conv)

# Perform deconvolution
output_deconv = deconv_layer(output_conv)
print("\nDeconvolution Output:")
print(output_deconv)

Convolution Output:
tensor([[[[-6.5565, -6.6727, -6.7889],
          [-7.1374, -7.2536, -7.3698],
          [-7.7183, -7.8345, -7.9507]]]], grad_fn=<ConvolutionBackward0>)

Deconvolution Output:
tensor([[[[-0.0909, -1.5911, -1.7132, -1.5599, -0.0114],
          [ 1.2751,  0.2821, -0.3465, -1.6067, -0.5722],
          [ 0.3417, -2.8264, -3.7623, -4.0925, -0.8312],
          [ 0.5782, -0.9540, -1.8161, -2.3609, -0.7837],
          [-1.1403, -3.6056, -3.8922, -2.6710, -0.1539]]]],
       grad_fn=<ConvolutionBackward0>)


In [12]:
# Define a convolutional block using nn.Sequential
conv_block = nn.Sequential(
    # Convolutional layer
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
    # Batch normalization layer
    nn.BatchNorm2d(64),
    # ReLU activation function
    nn.ReLU(inplace=True)
)
# Print the convolutional block
print(conv_block)
# Example input tensor (batch_size, channels, height, width)
input_tensor = torch.randn(1, 3, 32, 32)
# Forward pass through the convolutional block
output_tensor = conv_block(input_tensor)
# Print the shape of the output tensor
print(output_tensor.shape)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)
torch.Size([1, 64, 32, 32])


In [15]:
# Define a simple linear regression model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)  # 1 input feature, 1 output

    def forward(self, x):
        return self.linear(x)
# Create model, loss function, and optimizer
model = LinearRegression()
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # weights, bias, learning rate
# Dummy data (y = 2x + 1)
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y = torch.tensor([[3.0], [5.0], [7.0], [9.0]])
# Training loop
for epoch in range(100):
    # Forward pass
    predictions = model(x) # forward method is automatically called when you pass data into model
    loss = criterion(predictions, y) # MSE
    # Backward pass and optimization
    optimizer.zero_grad() # clear gradients from previous iteration
    loss.backward() # compute weights by backprop
    optimizer.step() # update weights and bias
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# Test the model
test_x = torch.tensor([[5.0]])
predicted_y = model(test_x)
print(f'Prediction for input {test_x.item()}: {predicted_y.item():.4f}')

Epoch [10/100], Loss: 1.6223
Epoch [20/100], Loss: 0.1157
Epoch [30/100], Loss: 0.0724
Epoch [40/100], Loss: 0.0673
Epoch [50/100], Loss: 0.0633
Epoch [60/100], Loss: 0.0596
Epoch [70/100], Loss: 0.0562
Epoch [80/100], Loss: 0.0529
Epoch [90/100], Loss: 0.0498
Epoch [100/100], Loss: 0.0469
Prediction for input 5.0: 10.6297


In [17]:
# Define a simple binary classification model
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.linear = nn.Linear(1, 1)  # 1 input feature, 1 output
        self.sigmoid = nn.Sigmoid()    # Sigmoid activation for binary classification

    def forward(self, x):
        return self.sigmoid(self.linear(x))

# Create model, loss function, and optimizer
model = BinaryClassifier()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Dummy data (y = 0 for x < 2.5, y = 1 for x >= 2.5)
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y = torch.tensor([[0.0], [0.0], [1.0], [1.0]]) # 0-> fake, 1-> real

# Training loop
for epoch in range(100):
    # Forward pass
    predictions = model(x)
    loss = criterion(predictions, y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# Test the model
test_x = torch.tensor([[2.0], [3.0]])
predicted_y = model(test_x)
print(f'Predictions for input {test_x.squeeze().tolist()}: {predicted_y.squeeze().tolist()}')

Epoch [10/100], Loss: 0.5192
Epoch [20/100], Loss: 0.4917
Epoch [30/100], Loss: 0.4757
Epoch [40/100], Loss: 0.4610
Epoch [50/100], Loss: 0.4473
Epoch [60/100], Loss: 0.4344
Epoch [70/100], Loss: 0.4223
Epoch [80/100], Loss: 0.4109
Epoch [90/100], Loss: 0.4002
Epoch [100/100], Loss: 0.3901
Predictions for input [2.0, 3.0]: [0.5055983662605286, 0.7108482122421265]


In [ ]:
# plotting losss
